Start this on the zedboard first
```bash
$ sudo python3 litex_server.py --devmem --devmem-offset 0x40000000
```

In [202]:
%pylab notebook
from struct import pack, unpack
import time
import sys
sys.path.append("../..")
from common import *
from scope_app import autoBitslip, autoIdelay

def hd(dat, word_width=1, offs=0):
    ''' print a hex-dump, word_width in bytes '''
    for i, d in enumerate(dat):
        if i % 8 == 0 and len(dat) > 8:
            print('\n{:04x}: '.format(i * word_width + offs), end='')
        print('{:0{ww}x} '.format(d, ww=word_width * 2), end='')
    print()

Populating the interactive namespace from numpy and matplotlib


# Setup SPI, reset and dump LTC2175 registers

In [618]:
r = conLitexServer('../build/csr.csv')
print("f_sample:", r.regs.lvds_f_sample_value.read() / 1e6, "MHz")

ltc_spi = LTC_SPI(r)
for i in range(5):
    print("{:2x}: {:02x}".format(i, ltc_spi.get_ltc_reg(i)))

ltc_spi.set_ltc_reg(0, 0x80)   # Software reset
# ltc_spi.set_ltc_reg(1, 0x10)   # Sleep mode
# ltc_spi.set_ltc_reg(1, 0x00)   # Waky waky

ltc_spi.setTp(1)
autoBitslip(r)
autoIdelay(r)

Connected to Port 1234
LTC2175 demonstrator 2019-09-13 18:54:36
f_sample: 125.000626 MHz
 0: ff
 1: 00
 2: 00
 3: 80
 4: 01
autoBitslip(): aligned after 6
autoIdelay(): min = 6, mean = 12, max = 19 idelays


# Play with bitslip / idelay parameters

### show status

In [595]:
def print_frm():
    idel = r.regs.lvds_idelay_value.read()
    v = r.regs.lvds_frame_peek.read()
    print("ID: {:}  F: {:08b}".format(idel, v))
    for i in range(4):
        v = getattr(r.regs, 'lvds_data_peek{:}'.format(i)).read()
        print("{:}: {:016b}".format(i, v))
ltc_spi.setTp(1)
print_frm()

ID: 12  F: 11110000
0: 0000000000000100
1: 0000000000000100
2: 0000000000000100
3: 0000000000000100


### bitslip until we get 0xF0 on frame_peek

In [113]:
r.regs.lvds_bitslip_csr.write(1)
print_frm()

ID: 13  F: 11110000
0: 0000000000000100
1: 0000000000000100
2: 0000000000000100
3: 0000000000000100


### IDELAY adjustments to find the eye limits

In [10]:
r.regs.lvds_idelay_inc.write(1)
print_frm()

ID: 19  F: 11110000
0: 0000000000010000
1: 0000000000000000
2: 0000000000000000
3: 0000000000000100


In [22]:
r.regs.lvds_idelay_dec.write(1)
print_frm()

ID: 7  F: 01111000
0: 0000000000000100
1: 0000000000000100
2: 0000000000000100
3: 0000000000000001


do a bitslip until we get 0x0F

### map data lanes
Check that each bit in the test-pattern end up in the right place in the output word

In [211]:
for ch in range(4):
    print("Channel {:d}".format(ch))
    for i in range(14):
        tpValue = 1 << i
        ltc_spi.setTp(tpValue)
        rVal = getattr(r.regs, 'lvds_data_peek{:d}'.format(ch)).read()
        print("{:016b} {:016b}".format(tpValue, rVal))    

Channel 0
0000000000000001 0000000000000100
0000000000000010 0000000000001000
0000000000000100 0000000000010000
0000000000001000 0000000000100000
0000000000010000 0000000001000000
0000000000100000 0000000010000000
0000000001000000 0000000100000000
0000000010000000 0000001000000000
0000000100000000 0000010000000000
0000001000000000 0000100000000000
0000010000000000 0001000000000000
0000100000000000 0010000000000000
0001000000000000 0100000000000000
0010000000000000 1000000000000000
Channel 1
0000000000000001 0000000000000100
0000000000000010 0000000000001000
0000000000000100 0000000000010000
0000000000001000 0000000000100000
0000000000010000 0000000001000000
0000000000100000 0000000010000000
0000000001000000 0000000100000000
0000000010000000 0000001000000000
0000000100000000 0000010000000000
0000001000000000 0000100000000000
0000010000000000 0001000000000000
0000100000000000 0010000000000000
0001000000000000 0100000000000000
0010000000000000 1000000000000000
Channel 2
0000000000000001 0

# Read ADC sample

In [212]:
ltc_spi.set_ltc_reg(3, 0)   # Test pattern off
ltc_spi.set_ltc_reg(1, 0)   # Randomizer off

In [213]:
# Lowest 2 bits are always zero in 16 bit LVDS mode
# 12 bit ADC, so next 2 bits are also zero
rVal = r.regs.lvds_data_peek0.read() >> 4
print("0x{0:03x} 0b{0:012b}".format(rVal))
# Decode offset binary mode
print(rVal - (1 << 11))

0x805 0b100000000101
5


# Read randomized ADC sample

In [ ]:
ltc_spi.set_ltc_reg(1, (1<<6))   # Randomizer on

In [ ]:
rVal = r.regs.lvds_data_peek.read() >> 4
print("0x{0:03x} 0b{0:012b}".format(rVal))
# XOR LSB with any other bit
if rVal & 0x01:
    rVal ^= 0xFFE
# Decode offset binary mode
print(rVal - (1 << 11))

# Read acquisition memory

In [225]:
ltc_spi.setTp(0x0 >> 2)

In [229]:
ltc_spi.setTp(0x1234 >> 2)

In [218]:
hex(r.regs.lvds_data_peek0.read())

'0x8050'

In [230]:
r.regs.acq_trig_csr.write(1)

In [232]:
hd(r.read(r.mems.sample0.base, 32), 2)


0000: 1234 1234 1234 1234 1234 1234 1234 1234 
0010: 1234 1234 1234 1234 1234 1234 1234 1234 
0020: 1234 1234 1234 1234 1234 1234 1234 1234 
0030: 1234 1234 1234 1234 1234 1234 1234 1234 


In [541]:
def big_read(r, offset, N, chunk_size=127):
    """ why doesn't RemoteClient take care of chunking for me ??? """
    # offset = 0x10000000
    # N = 0x0A00
    # chunk_size = 255
    dats = []
    while N > 0:
        temp = r.read(offset, min(chunk_size, N))
        offset += len(temp) * 4
        N -= len(temp)
        dats.append(temp)
    return hstack(dats)

In [589]:
r.regs.ctrl_reset.write(1)
r.regs.ctrl_bus_errors.read()

7927

In [588]:
hex(r.read(0x10000004))

'0x1234'

In [593]:
hex(4096 // 2)

'0x800'

In [582]:
hd(big_read(r, 0x10000000, 0x700))

KeyboardInterrupt: 

In [654]:
s_addr = 0x10000000
offs = 0x380
hd(r.read(s_addr + offs * 4, 0xF0), 1, s_addr + offs - 0x10000000)


0380: 380 381 382 383 384 385 386 387 
0388: 388 389 38a 38b 38c 38d 38e 38f 
0390: 390 391 392 393 394 395 396 397 
0398: 398 399 39a 39b 39c 39d 39e 39f 
03a0: 3a0 3a1 3a2 3a3 3a4 3a5 3a6 3a7 
03a8: 3a8 3a9 3aa 3ab 3ac 3ad 3ae 3af 
03b0: 3b0 3b1 3b2 3b3 3b4 3b5 3b6 3b7 
03b8: 3b8 3b9 3ba 3bb 3bc 3bd 3be 3bf 
03c0: 3c0 3c1 3c2 3c3 3c4 3c5 3c6 3c7 
03c8: 3c8 3c9 3ca 3cb 3cc 3cd 3ce 3cf 
03d0: 3d0 3d1 3d2 3d3 3d4 3d5 3d6 3d7 
03d8: 3d8 3d9 3da 3db 3dc 3dd 3de 3df 
03e0: 3e0 3e1 3e2 3e3 3e4 3e5 3e6 3e7 
03e8: 3e8 3e9 3ea 3eb 3ec 3ed 3ee 3ef 
03f0: 3f0 3f1 3f2 3f3 3f4 3f5 3f6 3f7 
03f8: 3f8 3f9 3fa 3fb 3fc 3fd 3fe 3ff 
0400: ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff 
0408: ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff 
0410: ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff 
0418: ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff 
0420: ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff

In [607]:
r.regs.ctrl_bus_errors.read()

64